In [1]:
# import library
import os
import numpy as np
import pandas as pd
from pytesseract import pytesseract
from sklearn.model_selection import train_test_split
from sys import platform
import cv2
from Levenshtein import distance as levenshtein_distance
from model_num_extractor.easyocr_extractor import easyocr_mn_extractor
from logger import get_logger
import easyocr
import time

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
C:\Users\rrai34\Anaconda3\lib\site-packages\torchvision\models\_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will be removed in 0.15. Please access them via the appropriate Weights Enum instead.
  warnings.warn(
C:\Users\rrai34\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\rrai34\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
# Required to run easyocr in windows - not sure why.
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# starting up easyocr
reader = easyocr.Reader(['en'])

In [5]:
# import logger
logger = get_logger("easyocr-extractor-app")

In [6]:
# read in data
df = pd.read_csv('./new_data/product_model_serial_numbers.csv')
df.head()

,SERVICE_JOB_ID,FILE_NAME,example,PRODUCT_LINE,MODEL_NUMBER,SERIAL_NUMBER
0,SJ48132474,image-pre-inspection-proof-of-purchase-72fddee...,data/Laundry/SJ48132474/image-pre-inspection-p...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M
1,SJ48132474,image-pre-inspection-proof-of-purchase-bcd8901...,data/Laundry/SJ48132474/image-pre-inspection-p...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M
2,SJ48132474,image-pre-inspection-serial-number-a2dc2e0b-7a...,data/Laundry/SJ48132474/image-pre-inspection-s...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M
3,SJ48132474,image-pre-inspection-serial-number-d843dfb6-fb...,data/Laundry/SJ48132474/image-pre-inspection-s...,HOME LAUNDRY GAS,DVG54R7600C/A3,0B1T5BCNC00144M
4,SJ48144276,image-pre-inspection-proof-of-purchase-8e82720...,data/Laundry/SJ48144276/image-pre-inspection-p...,HOME LAUNDRY GAS,GTD42GASJ2WW,GTD42GASJ2WW


In [8]:
df.describe()

,SERVICE_JOB_ID,FILE_NAME,example,PRODUCT_LINE,MODEL_NUMBER,SERIAL_NUMBER
count,3347,3347,3347,2429,2425,2424
unique,1573,3309,3309,4,358,1141
top,SJ48595774,image-pre-inspection-serial-number-105d636a-08...,data/Laundry/SJ48547655/image-pre-inspection-s...,DISHWASHER,DW80R2031US/AA,0ERBGDAK201631J
freq,11,4,4,1031,169,11


Get rid of data points which doesn't have model number.

In [9]:
df0 = df.dropna(subset=['MODEL_NUMBER', ])
df0.describe()

,SERVICE_JOB_ID,FILE_NAME,example,PRODUCT_LINE,MODEL_NUMBER,SERIAL_NUMBER
count,2425,2425,2425,2425,2425,2424
unique,1172,2387,2387,4,358,1141
top,SJ48595774,image-pre-inspection-serial-number-105d636a-08...,data/Laundry/SJ48547655/image-pre-inspection-s...,DISHWASHER,DW80R2031US/AA,0ERBGDAK201631J
freq,11,4,4,1031,169,11


Checking how many unique service id's the dataframe has.

In [10]:
len(df0.SERVICE_JOB_ID.unique())

1172

Group the services by job id and create a new column `files` that holds all the images path as a list. Each service id is treated as a data point.

In [11]:
df_files = df0.groupby(['SERVICE_JOB_ID',])['example'].agg(files=list).reset_index()
df_files.head()

,SERVICE_JOB_ID,files
0,SJ48132474,[data/Laundry/SJ48132474/image-pre-inspection-...
1,SJ48144276,[data/Laundry/SJ48144276/image-pre-inspection-...
2,SJ48148704,[data/Laundry/SJ48148704/image-pre-inspection-...
3,SJ48155130,[data/Laundry/SJ48155130/image-pre-inspection-...
4,SJ48165170,[data/Laundry/SJ48165170/image-pre-inspection-...


In [12]:
df_mn = df0.groupby(['SERVICE_JOB_ID', 'MODEL_NUMBER']).size().reset_index(name='count')
df_mn.drop(['count'], axis=1, inplace=True)
df_mn.head()

,SERVICE_JOB_ID,MODEL_NUMBER
0,SJ48132474,DVG54R7600C/A3
1,SJ48144276,GTD42GASJ2WW
2,SJ48148704,GTX22GASK0WW
3,SJ48155130,GTD33GASK0WW
4,SJ48165170,00000000


In [13]:
df1 = df_files.join(df_mn.set_index('SERVICE_JOB_ID'), on="SERVICE_JOB_ID")
df1.head()

,SERVICE_JOB_ID,files,MODEL_NUMBER
0,SJ48132474,[data/Laundry/SJ48132474/image-pre-inspection-...,DVG54R7600C/A3
1,SJ48144276,[data/Laundry/SJ48144276/image-pre-inspection-...,GTD42GASJ2WW
2,SJ48148704,[data/Laundry/SJ48148704/image-pre-inspection-...,GTX22GASK0WW
3,SJ48155130,[data/Laundry/SJ48155130/image-pre-inspection-...,GTD33GASK0WW
4,SJ48165170,[data/Laundry/SJ48165170/image-pre-inspection-...,00000000


In [14]:
df1.describe()

,SERVICE_JOB_ID,files,MODEL_NUMBER
count,1172,1172,1172
unique,1172,1172,358
top,SJ48132474,[data/Laundry/SJ48132474/image-pre-inspection-...,DW80R2031US/AA
freq,1,1,71


In [18]:
X = df1.iloc[100:, :-1]
y = df1.loc[100:, 'MODEL_NUMBER']

Divide into train test set

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75)

Create a `MN` column and initialize it with `None` value. This column will store the predicted model number.

In [20]:
def predict_mn(df, rotate_pic = False):
    df.reset_index(drop=True, inplace=True)
    pred = [None, ]*len(df)
    for i, row in df.iterrows():
        mod_nums = []
        files = row['files']
        for fn in files:
            mn = easyocr_mn_extractor(fn, logger, rotate_pic)
            if mn is not None:
                mod_nums.append(mn)
        if (i % 100 == 0):
            print(f"Processed {i}th datapoint...")
        if len(mod_nums) > 0:
            pred[i] = (max(mod_nums, key=len))
    return pred

In [21]:
def get_accuracy(given, predicted, ld=5):
    accurate = 0
    for i in range(len(given) - 1):
        if predicted[i] is not None:
            if levenshtein_distance(predicted[i], given[i]) <= ld:
                accurate += 1
    return round(accurate*100/len(given), 2)

In [22]:
# use train test to predict and then get accuracy
# train_pred = predict(X_train)
# get_accuracy(y_train.to_list(), train_pred)

In [23]:
# test_pred = predict(X_test)
# get_accuracy(y_test.to_list(), test_pred)

In [24]:
pd.options.mode.chained_assignment = None
df1['MN'] = None
df1.head()

,SERVICE_JOB_ID,files,MODEL_NUMBER,MN
0,SJ48132474,[data/Laundry/SJ48132474/image-pre-inspection-...,DVG54R7600C/A3,None
1,SJ48144276,[data/Laundry/SJ48144276/image-pre-inspection-...,GTD42GASJ2WW,None
2,SJ48148704,[data/Laundry/SJ48148704/image-pre-inspection-...,GTX22GASK0WW,None
3,SJ48155130,[data/Laundry/SJ48155130/image-pre-inspection-...,GTD33GASK0WW,None
4,SJ48165170,[data/Laundry/SJ48165170/image-pre-inspection-...,00000000,None


In [25]:
# %%timeit
start_time = time.time()

# using model to extract model number for all service_job_id
for i, row in df1.iterrows():
    mod_nums = []
    files = row['files']
    for fn in files:
        mn = easyocr_mn_extractor(fn, logger)
        if mn is not None:
            mod_nums.append(mn)
    if (i % 100 == 0):
        print(f"Printing model number for {i} datapoint...")
    if len(mod_nums) > 0:
#         print(max(mod_nums, key=len))
        df1.loc[i, 'MN'] = max(mod_nums, key=len)
print(f"time taken: {time.time() - start_time} s")

Printing model number for 0 datapoint...
Printing model number for 100 datapoint...
Printing model number for 200 datapoint...
Printing model number for 300 datapoint...
Printing model number for 400 datapoint...
Printing model number for 500 datapoint...
Printing model number for 600 datapoint...
Printing model number for 700 datapoint...
Printing model number for 800 datapoint...
Printing model number for 900 datapoint...
Printing model number for 1000 datapoint...
Printing model number for 1100 datapoint...
time taken: 17524.430861473083 s


In [26]:
df1.head()

,SERVICE_JOB_ID,files,MODEL_NUMBER,MN
0,SJ48132474,[data/Laundry/SJ48132474/image-pre-inspection-...,DVG54R7600C/A3,None
1,SJ48144276,[data/Laundry/SJ48144276/image-pre-inspection-...,GTD42GASJ2WW,None
2,SJ48148704,[data/Laundry/SJ48148704/image-pre-inspection-...,GTX22GASK0WW,GTX22GASKGLW
3,SJ48155130,[data/Laundry/SJ48155130/image-pre-inspection-...,GTD33GASK0WW,GTD33GASKOLH
4,SJ48165170,[data/Laundry/SJ48165170/image-pre-inspection-...,00000000,None


In [27]:
given = df1['MODEL_NUMBER'].to_list()
pred = df1['MN'].to_list()

In [28]:
accurate = 0
for i in range(len(given) - 1):
#     print("pred = ", pred[i], " given = ", given[i])
    if pred[i] is not None:
        if levenshtein_distance(pred[i], given[i]) <= 5:
#             print(f"Given = {given[i]}, \t\tPred = {pred[i]}")
            accurate += 1

In [29]:
# number of accurate model numbers
accurate

109

In [30]:
# accuracy as percentage
accuracy = round(accurate*100/len(given), 2)
accuracy

9.3